# Homework №1
## Problem 1

In [1]:
%%capture
!pip3 install qiskit #installing necessary packages

In [38]:
import numpy as np
import qiskit as qk
from sympy.physics.quantum import TensorProduct
from sympy import *

In [39]:
with open('token', 'r') as token_file:
    token = token_file.read() #importing token file

In [4]:
%%capture
qk.IBMQ.save_account(token, overwrite = True)
qk.IBMQ.load_account()

In [5]:
provider = qk.IBMQ.get_provider(hub = 'ibm-q')

In [6]:
devices = provider.backends(filters=lambda x: (3 <= x.configuration().n_qubits <= 5) and not x.configuration().simulator)

In [7]:
devices #List of available hardware

[<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [8]:
%%capture
real_backend = qk.providers.ibmq.least_busy(devices)
print(real_backend.configuration().n_qubits) #selecting least busy machine

__init__.least_busy:WARNING:2020-10-15 15:36:18,995: Unable to find backend reservation information. It will not be taken into consideration. string indices must be integers
__init__.least_busy:WARNING:2020-10-15 15:36:19,795: Unable to find backend reservation information. It will not be taken into consideration. string indices must be integers
__init__.least_busy:WARNING:2020-10-15 15:36:20,582: Unable to find backend reservation information. It will not be taken into consideration. string indices must be integers
__init__.least_busy:WARNING:2020-10-15 15:36:21,088: Unable to find backend reservation information. It will not be taken into consideration. string indices must be integers
__init__.least_busy:WARNING:2020-10-15 15:36:22,211: Unable to find backend reservation information. It will not be taken into consideration. string indices must be integers
__init__.least_busy:WARNING:2020-10-15 15:36:22,992: Unable to find backend reservation information. It will not be taken into con

In [9]:
simd_backend = qk.Aer.get_backend('qasm_simulator')

## Problem 2

Assuming wawefunction is following:
$\vert \psi \rangle = \sin \theta \vert 0 \rangle + \exp{i\varphi} \cos \theta \vert 1 \rangle$
We want to measure $\varphi

Actually we measure projection on $Z$ - axis that corresponds $\theta$ angle. Thus $\varphi$ angle describes projection onto $X,~Y$ axes. To measeure $\varphi$ we have to rotate system (or alternavely Vector) in the following way: $X,~Y,~Z \rightarrow Z,~X,~Y$. Obviously guiding vector of axis of rotation is $(1 1 1)$ and angle to rotate is $\alpha = 2 \pi / 3$.
To make rotation itself we should compute corresponding axis. We know the form of such rotations :
$$\hat{R}_{\vec{n}} (\alpha)= e^{-i \vec{n} \cdot \hat{\vec{\sigma}} \alpha / 2}$$
Since all sigma's satisfy $\hat{\sigma}_i^2 = \hat{I}$ it can be rewritten in the following form: 

$$\hat{R}_{\vec{n}} (\alpha) = I \cos{\frac{\alpha}{2}} - i \vec{n} \cdot \hat{\vec{\sigma}} \sin{\frac{\alpha}{2}}$$

Thus the function to compute such operator is:

In [10]:
sigmax = np.array([[0, 1 ], [1 , 0]], dtype = 'complex128') #definitions of necessary matrices
sigmay = np.array([[0,-1j], [1j, 0]], dtype = 'complex128')
sigmaz = np.array([[1, 0 ], [0 ,-1]], dtype = 'complex128')
I = np.array([[1, 0 ], [0 , 1]], dtype = 'complex128')

In [15]:
def rot_mat(in_vector, angle):
    vector = np.array(in_vector, dtype = 'float64')
    vector = vector / np.linalg.norm(vector) #normalising guiding vector to preserve norm of wavefunction
    matrix = vector[0] * sigmax + vector[1] * sigmay + vector[2] * sigmaz
    result = I * np.cos(angle / 2) - 1j * matrix * np.sin(angle / 2)
    return result

In [16]:
rot = rot_mat([1, 1, 1], 2 * np.pi / 3) #Now, we have the form of rotation operator, that makes phi available for measurment
print(rot)

[[ 0.5-0.5j -0.5-0.5j]
 [ 0.5-0.5j  0.5+0.5j]]


In the next code block we actually measure $\varphi$. To perform it we initialise random state in form: 
$$\vert \psi \rangle = e^{i\psi} \sin \theta \vert 0 \rangle + e^{i\phi} \cos \theta \vert 1 \rangle$$
But it can be rewritten:
$$\vert \psi \rangle = e^{i\psi} (\sin \theta \vert 0 \rangle + e^{i\varphi} \cos \theta \vert 1 \rangle)$$ 
Where $\varphi = \phi - \psi$
After some algebraic transformations we can get a formula, where probabilities and $/varphi$ are connected:
$$\frac{p(\vert 0 \rangle)}{p(\vert 1 \rangle)} = - \frac{e^{i\varphi} (\sin{2\theta} \sin{\varphi} + 1)}{e^{i\varphi} (\sin{2\theta} \sin{\varphi} - 1)} = r$$
Thus the output is:
$$\sin{\varphi} \sin{2 \theta} = \frac{r - 1}{r + 1}$$

Now we move to actual measurment of $\varphi$

In [17]:
state = np.random.rand(2) + 1j * np.random.rand(2) #definition of random state
state = state / np.linalg.norm(state)
psi_angle = np.angle(state[0])
state = np.exp(-1j * psi_angle) * state
varphi = np.angle(state[1])
theta  = np.arcsin(state[0])
print(varphi, theta)
q = qk.QuantumRegister(1) #Initialising one cubit state,implementing our rotation operator and measuring final state
c = qk.ClassicalRegister(1)
qc = qk.QuantumCircuit(q, c)
qc.initialize(state, q)
qc.unitary(rot, q)
qc.measure(q, c)
res = qk.execute(qc, backend = simd_backend, shots = 65536).result()
r_simd = res.get_counts()
print(r_simd)
qc.draw()
r = r_simd['0'] / r_simd['1'] #computing phi
sin_phi = (r - 1) / (r + 1) / np.sin(2 * theta)
res_phi = np.arcsin(sin_phi)
print(res_phi)

0.8073073837667796 (0.48524430609782726+0j)
{'0': 52195, '1': 13341}
(0.8016192569928041+0j)


## Problem 3

We have to prepare exact state from previous task using finite number of standart gates. So at first we have to define these operators.

In [18]:
H = np.array([[1, 1], [1, -1]], dtype = 'complex128') / np.sqrt(2)
S = np.array([[1, 0], [0, 1j]], dtype = 'complex128')
Hadamard = Matrix([[1, 1], [1, -1]]) / sqrt(2)

We get three angles $\alpha, \beta, \gamma$. Steps of Euler rotation are:

1. Perform $\hat{R}_z (\gamma)$
2. Perform $\hat{R}_x (\beta )$
3. Perform $\hat{R}_z (\alpha)$

To do so we need to express $\hat{R}_x$ as combination of $\hat{H},\hat{S},\hat{X}$ and $\hat{R}_z$. We will use the following form: $ \hat{R}_x = \hat{H} \hat{R}_z \hat{H}$.But is not the only form odf such decomposition

In [19]:
R_x = cos(theta / 2) * eye(2) - I * sin(theta / 2) * sigmax #cecking previous words
R_z = cos(theta / 2) * eye(2) - I * sin(theta / 2) * sigmaz
custom_Rx = Hadamard @ R_z @ Hadamard

Now we can write our own $\hat{U}_3$ gate:

In [20]:
def custom_R_x(qc, qr, alpha):
    qc.h(qr)
    qc.R_z(alpha, qr)
    qc.h(qr)
def custom_u3(qc, qr, alpha, beta, gamma):
    qc.R_z(beta, qr)
    custom_R_x(qc, qr, - np.pi / 2)
    qc.R_z(alpha, qr)
    custom_R_x(qc, qr,   np.pi / 2)
    qc.R_x(gamma, qr)
def rot_Matrix_ZX(vec_z, vec_x = [1, 0, 0]):
    vec_z, vec_x = np.array(vec_z), np.array(vec_x)
    normal_x = vec_x / np.linalg.norm(vec_x)
    normal_z = vec_z / np.linalg.norm(vec_z)
    normal_y = np.cross(normal_z, normal_x)
    return np.array([normal_x, nnormal_y, normal_z])
def _angle_From_Sin_Cos(sin_v, cos_v):
    sin_v, cos-V = float(sin_v), float(cos_v)
    if cos_v > 0:
        return np.arctan(sin_v / cos_v)
    elif cos_v < 0:
        return - np.arctan(- sin_v / cos_v)
    else:
        return np.sign(sin_v) * np.pi / 2
def rot_Matrix_Euler(matrix): #Working with matrix in form [[aa,ab,...]]
    cb = matrix[2, 2]
    if abs(cb) == 1:
        sb = 0
        cc = 1
        sc = 0
        sa = matrix[1, 0]
        ca = matrix[0, 0]
    else:
        sb = sqrt(1 - cb**2)
        cc = matrix[0, 2] / sb
        sc = matrix[1, 2] / sb
        ca = - matrix[2, 0] / sb
        sa = matrix[2, 1] / sb
    return np.array([_angle_From_Sin_Cos(sa, ca), _angle_From_Sin_Cos(sb, cb), _angle_From_Sin_Cos(sc, cc)])
def vec_From_Angles(theta, phi):
    return np.array([np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)])

SyntaxError: cannot assign to operator (<ipython-input-20-13e938ae237e>, line 18)

In [21]:
vec = vec_From_Angles(np.pi / 3, np.pi / 3) #trying to create necessary state
mat = rot_Matrix_ZX(vec)
ans = rot_Matrix_Euler(mat)

NameError: name 'vec_From_Angles' is not defined

In [22]:
q = qk.QuantumRegister(1) #Returning to quantum machine
c = qk.ClassicalRegister(1)
qc = qk.QuantumCircuit(q, c)
qc.rz(ans[1], q[0])
qc.h(q[0])
qc.rz(- np.pi / 2, q[0])
qc.h(q[0])
qc.rz(ans[0],  q[0])
qc.h(q[0])
qc.rz(np.pi / 2, q[0])
qc.h(q[0])
qc.rz(ans[2], q[0])
qc.measure(q, c)
res = qk.execute(qc, backend = simd_backend, shots = 65536).result()
r_simd = res.get_counts()
print(r_simd)
qc.draw()
res_theta = np.arctan(np.sqrt(r_simd['0'] / r_simd['1']))
print(res_theta, res_theta - np.pi / 3)

NameError: name 'ans' is not defined

In [23]:
q = qk.QuantumRegister(1)   #comparison with theoretical result
c = qk.ClassicalRegister(1)
qc = qk.QuantumCircuit(q, c)
qc.rz(ans[1],     q[0])
qc.h(q[0])
qc.rz(- np.pi / 2, q[0])
qc.h(q[0])
qc.rz(ans[0],  q[0])
qc.h(q[0])
qc.rz(np.pi / 2, q[0])
qc.h(q[0])
qc.rz(ans[2],  q[0])
rot = rot_mat([1, 1, 1], 2 * np.pi / 3)
qc.unitary(rot, q)
qc.measure(q, c)
res = qk.execute(qc, backend = simd_backend, shots = 65536).result()
r_simd_v0 = res.get_counts()
rat = r_simd_v0['0'] / r_simd_v0['1']
comp_rat = - (np.sin(2 * res_theta) * sin(np.pi / 3) + 1.) / (np.sin(2 * res_theta) * sin(np.pi / 3) - 1.)
ratio_of_ratios = rat / comp_rat
print(ratio_of_ratios)

NameError: name 'ans' is not defined

## Problem 4

In [24]:
Hadamard = Matrix([[1, 1], [1, -1]])/sqrt(2)
X = Matrix([[0, 1], [1, 0]])
Z = Matrix([[1, 0], [0, -1]])


1)

In [25]:
Hadamard @ X @ Hadamard - Z

Matrix([
[0, 0],
[0, 0]])

2)

cZ matrix by definition is:

$$  \begin{bmatrix}
    1 & 0 & 0 & 0\\
    0 & 1 & 0 & 0\\
    0 & 0 & 1 & 0\\
    0 & 0 & 0 &-1
    \end{bmatrix}    $$
    
So, as we can see it is symmetric symmetric with respect to permutations of q-bits.


3)

The definition of Hadamard operator for $m$ independent q-bits is:
$$ \hat H_m = I_m * \hat H_1 $$

In [26]:
H_2_1 = TensorProduct(eye(2), Hadamard) #For the first q-bit
H_2_2 = TensorProduct(Hadamard,eye(2)) #For the second one

In [27]:
H_2_1

Matrix([
[sqrt(2)/2,  sqrt(2)/2,         0,          0],
[sqrt(2)/2, -sqrt(2)/2,         0,          0],
[        0,          0, sqrt(2)/2,  sqrt(2)/2],
[        0,          0, sqrt(2)/2, -sqrt(2)/2]])

In [28]:
H_2_2

Matrix([
[sqrt(2)/2,         0,  sqrt(2)/2,          0],
[        0, sqrt(2)/2,          0,  sqrt(2)/2],
[sqrt(2)/2,         0, -sqrt(2)/2,          0],
[        0, sqrt(2)/2,          0, -sqrt(2)/2]])

The definition of cX for th second q-bit matrix is:

$$  \begin{bmatrix}
    1 & 0 & 0 & 0\\
    0 & 0 & 0 & 1\\
    0 & 0 & 1 & 0\\
    0 & 1 & 0 & 0
    \end{bmatrix}    $$

In [29]:
cX_2 = Matrix([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]])

In [30]:
rc = H_2_1 @ H_2_2 @ cX_2 @ H_2_1 @ H_2_2 #The circuit itself

In [31]:
rc 

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 0, 1],
[0, 0, 1, 0]])

In [32]:
cnot = Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]) #definition of cX for the first q-bit

In [33]:
rc - cnot

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

4)

We need to find controlled $e^{i\alpha}$ gate. It should work as: multiply both components 
of the second cubit on $e^{i\alpha}$ if the first one is in $\vert 1 \langle$ state. Thus the operator's matrix is the following:

$$  \begin{bmatrix}
    1 & 0 & 0 & 0\\
    0 & e^{i\alpha} & 0 & 0\\
    0 & 0 & 1 & 0\\
    0 & 0 & 0 &e^{i\alpha}
    \end{bmatrix}    $$
    
Obviously it's just another representation of the following tensor product:

$\hat{CExp} = I^{(1)}_{2} \otimes \hat{U}_1 (\alpha)$ and that corresponds just $U_1$ gate on the first qubit (see Sub-Task 3).

## Problem 5

If state is represented as $a \vert 0 \rangle \otimes \vert 0 \rangle + b \vert 1 \rangle \otimes \vert 0 \rangle + 
c \vert 0 \rangle \otimes \vert 1 \rangle + \vert 1 \rangle \otimes \vert 1 \rangle$ we can consider that in case of unentanglement it can be written $(\alpha \vert 0 \rangle + \beta \vert 1 \rangle) \otimes (\gamma \vert 0 \rangle + \delta \vert 1 \rangle)$. Thus we can write the following system:

$$
\begin{aligned} 
    \alpha \gamma = a & &\alpha \delta = b \\
    \beta \gamma  = c & &\beta \delta  = d 
\end{aligned}
$$

The criteria of solvable system in this case can be written as: $a d - b c = 0$. In this case this system is unentangled.

1)

Wavefunction is $\frac{2}{3} \vert 0 0 \rangle + \frac{1}{3} \vert 01 \rangle -  \frac{2}{3} \vert 11 \rangle$.

In [34]:
matrix = Matrix([[2 / 3, 0], [1 / 3, - 2 / 3]])
det = matrix.det()
print(det)

-0.444444444444444


This state is entangled, as we can see from system of equations, $\alpha$ or $\delta$ have to be zero, but coefficients $a$ and $b$ are net zeros, so this system is inconsistent

2)

Wavefunction is $\frac{1}{2} \vert 0 0 \rangle + \frac{i}{2} \vert 10 \rangle - 
\frac{i}{2} \vert 01 \rangle + \frac{1}{2} \vert 11 \rangle$.

In [42]:
matrix = Matrix([[1, 1j], [-1j, 1]]) / 2
det = matrix.det()
print(det)

0


This state is unentangled. obviously the solution for this system is $(-\frac{\sqrt(2)}{2}, \frac{i \sqrt(2)}{2}, -\frac{ \sqrt(2)}{2}, -\frac{i \sqrt(2)}{2})$

3)

Wavefunction is $\frac{1}{2} \vert 0 0 \rangle + \frac{1}{2} \vert 10 \rangle - 
\frac{1}{2} \vert 01 \rangle + \frac{1}{2} \vert 11 \rangle$.

In [44]:
matrix = Matrix([[1/2, 1/2], [-1/2, 1/2]]) / 2
det = matrix.det()
print(det)

0.125000000000000


This state is entangled. Here, we can also see inconsistent system.